In [ ]:
import pandas as pd
import requests
import time
from datetime import date
pd.set_option('display.max_columns', None)
import os

from utils import *


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


In [ ]:
os.getenv('data_access_token')


# Branches

In [196]:
last_page = call_foodics('branches', 1, return_last_page=True)

list_responses = call_foodics('branches', last_page)

1
page 1
branches?page=1
1
page 1
branches?page=1


In [ ]:
df_branches = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_branches

In [ ]:

# # convert this df_branches[['id', 'name']] to a dictionary of {id: name}
# dict_branches = df_branches.set_index('id')['name'].to_dict()
# dict_branches

In [ ]:
df_branches = df_branches[['id', 'name', 'opening_from', 'opening_to']]

In [ ]:
df_branches = df_branches.rename(columns={'id': 'branch_id', 'name': 'branch_name'})

# Products

In [197]:
last_page = call_foodics('products', 1, return_last_page=True)

list_responses = call_foodics('products', last_page, includables='category')

page 1
products?page=1
page 1
products?page=1&include=category
page 2
products?page=2&include=category
page 3
products?page=3&include=category
page 4
products?page=4&include=category
page 5
products?page=5&include=category
page 6
products?page=6&include=category
page 7
products?page=7&include=category
page 8
products?page=8&include=category


In [198]:
df_products = pd.DataFrame([item for sublist in list_responses for item in sublist])

In [199]:
df_products

,category,id,sku,barcode,name,name_localized,description,description_localized,image,is_active,is_stock_product,is_ready,pricing_method,selling_method,costing_method,preparation_time,price,cost,calories,created_at,updated_at,deleted_at,meta
0,"{'id': '96ecc1b0-0590-4e3d-9325-91b371c2a18a',...",964fde65-52d5-491c-b227-da1603227e82,MER-013,None,Coffee Bean 250g (Delivery) - F,None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,1,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18,None
1,"{'id': '96ecc1b0-0590-4e3d-9325-91b371c2a18a',...",964fde65-55c7-4cc3-84df-285efec97d70,MER-015,None,Matte Mug (Delivery) - F,None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,1,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None
2,"{'id': '964fde5b-9a21-44db-9dab-b98ff6a4f3d0',...",964fde65-57c6-450f-b537-38b8a3395ee0,sk-004,None,Hot chocolate-F (Hot chocolate-F),None,None,None,None,True,False,True,1,1,2,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None
3,"{'id': '964fde5b-55b6-4d27-af90-22bd41e2f54e',...",964fde65-5969-4b44-9a47-00849d643cb8,sk-007,None,Calibration (Calibration),None,None,None,None,True,False,True,1,1,2,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None
4,"{'id': '964fde5b-9a21-44db-9dab-b98ff6a4f3d0',...",964fde65-5b14-4fee-a63d-757f5a46dce0,sk-005,None,Iced Lotus Latte-F (SUPER),None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,2,NaN,0.0,NaN,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,"{'id': '97161ba2-e1ed-4711-8d43-3f0e71acc13c',...",98bd655e-1585-4004-994f-4daebfbd8a74,sk-0420,None,Playing Cards,None,None,None,None,True,False,True,1,1,2,NaN,5.0,NaN,None,2023-03-21 09:49:29,2023-03-21 09:50:20,None,None
351,"{'id': '98ad494c-da3f-404c-a63e-1d13bd1cf3d1',...",98bd8f0f-bb1c-4579-986a-55d0a230426d,sk-0423,None,Rice Pudding رز بحليب,None,None,None,None,True,False,True,1,1,2,NaN,2.0,NaN,None,2023-03-21 11:46:04,2023-03-21 11:49:02,None,None
352,"{'id': '98ad4709-821b-4bb6-8c8e-0157f7510d20',...",98bf9b53-3bc9-4715-9626-4c37476995c0,sk-0424,None,Playing Cards (Del),None,None,None,None,True,False,True,1,1,2,NaN,6.0,NaN,None,2023-03-22 12:12:00,2023-03-22 12:12:14,None,None
353,"{'id': '97161ba2-e1ed-4711-8d43-3f0e71acc13c',...",98cf591d-4deb-4242-9145-de3ee98be981,sk-0426,None,Buy 1 Get 1 Free (Mugs),None,None,None,None,True,False,True,1,1,2,NaN,10.0,NaN,None,2023-03-30 08:00:07,2023-03-30 08:00:32,None,None


In [ ]:
category_name_col = os.getenv('category_name_col')

df_products['category_name'] = df_products['category'].apply(lambda x: x[category_name_col])
df_products = df_products[~df_products.category_name.isna()]
df_products['category_name'] = df_products['category_name'].apply(lambda x: x.capitalize())

df_products['category_id'] = df_products['category'].apply(lambda x: x['id'])
df_products.rename(columns={'id': 'product_id', 'name_localized': 'product_name', 'sku':"product_sku"}, inplace=True)
df_products.drop('category', axis=1, inplace=True)
# df_products = df_products[df_products.deleted_at.isna()]
df_products

In [ ]:
df_products.isnull().sum()

In [ ]:
df_products = df_products[['product_id', 'product_sku', 'name', 'category_name', 'category_id', 'is_active', 'is_stock_product', 'price']]

In [ ]:
df_products = df_products.rename(columns={'name': 'product_name'})

In [ ]:
df_products.head()

In [ ]:
df_products.product_name.unique()

# Categories

In [ ]:
last_page = call_foodics('categories', 1, return_last_page=True)


list_responses = call_foodics('categories', last_page)

In [ ]:
df_categories = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_categories

In [ ]:
cats_to_be_deleted = df_categories[~df_categories.deleted_at.isna()]
cats_to_be_deleted

In [ ]:
df_categories = df_categories[df_categories.deleted_at.isna()]

In [ ]:
df_categories = df_categories[['id', category_name_col]].rename(columns={"id":"category_id", category_name_col:"category_name"})


In [ ]:
df_categories

In [ ]:
df_categories.category_name = df_categories.category_name.apply(lambda x: x.capitalize())

In [ ]:
print(df_categories['category_name'].tolist())

# Payment Method

In [ ]:
last_page = call_foodics('payment_methods', 1, return_last_page=True)

list_responses = call_foodics('payment_methods', last_page)

In [ ]:
df_payment_methods = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_payment_methods

In [ ]:
df_payment_methods = df_payment_methods[['id', 'name']].rename(columns={"id":"payment_method_id", 'name':"payment_method_name"})
df_payment_methods

In [ ]:
df_payment_methods.to_csv('../../data/payment_methods.csv', index=False)

# Orders

In [200]:
filter = {}



if os.path.exists('../../data/orders_final_include.csv'):
    print('orders_final_include.csv already exists')
    df_orders = pd.read_csv('../../data/orders_final_include.csv')
    df_orders.reset_index(drop=True, inplace=True)  
else:
    business_name = os.getenv('business_name')
    if business_name == 'Blk Coffee':
        print(f"Yes it is {business_name}")
        filter = {"business_date_after": "2023-05-01"}
        
    print('orders_final_include.csv does not exist')
    includables = 'products,branch,products.product,tags,products.promotion,payments.payment_method'
    # call the foodics api to get the orders
    last_page = call_foodics('orders', last_page, includables=includables, filter=filter, return_last_page=True)
    print('last_page: ', last_page)
    
    list_responses = call_foodics('orders', last_page, includables=includables, filter=filter)
    df_orders = pd.DataFrame([item for sublist in list_responses for item in sublist])
    df_orders
    df_orders.to_csv('../../data/orders_final_include.csv', index=False)

# TODO: remove this line after testing
df_orders = pd.read_csv('../../data/orders_final_include.csv')
df_orders.reset_index(drop=True, inplace=True)
    

Yes it is Blk Coffee
orders_final_include.csv does not exist
page 1
orders?page=1&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01
last_page:  2009
page 1
orders?page=1&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01
page 2
orders?page=2&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01
page 3
orders?page=3&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01
page 4
orders?page=4&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01
page 5
orders?page=5&include=products,branch,products.product,tags,products.promotion,payments.payment_method&filter[business_date_after]=2023-05-01


KeyboardInterrupt: 

In [ ]:
df_orders.created_at = pd.to_datetime(df_orders.created_at)
max_date = df_orders.created_at.max().date().strftime('%Y-%m-%d')
max_date

In [ ]:
today = date.today().strftime('%Y-%m-%d')
today

In [ ]:
df_orders=df_orders.reset_index(drop=True)

In [ ]:
df_orders.head()

In [ ]:
df_orders['branch'] = df_orders['branch'].astype(str)
df_orders['tags'] = df_orders['tags'].astype(str)
df_orders['payments'] = df_orders['payments'].astype(str)
df_orders['products'] = df_orders['products'].astype(str)


In [ ]:
df_orders['branch_id'] = df_orders['branch'].apply(lambda x: eval(x)['id'])


In [ ]:
orders_header = df_orders[['id', 'branch_id', 'created_at', 'total_price', 'source', 'type', 'status']].rename(columns={'id':'order_id', 'created_at':'order_date', 'total_price':'order_total_price', 'source':'order_source', 'type':'order_type', 'status':'order_status'})
orders_header.isnull().sum()

In [ ]:
orders_header.head()

# Order Details

In [ ]:
df_orders.head()

In [ ]:
df_products

In [ ]:
# I want to create a dataframe with the following columns:
# order_id, product_id, category_id, quantity, price

# I will create a list of dictionaries, where each dictionary is a row in the dataframe
# I will then convert the list of dictionaries to a dataframe

need_unavailable_category = False

list_order_details = []

for index, row in df_orders.iterrows():

    order_id = row['id']
    branch_id = eval(row['branch'])['id']
    order_date = row['created_at']
    order_total_price = row['total_price']
    order_source = row['source']
    order_type = row['type']
    order_status = row['status']

    for order_product in eval(row['products']):

        product_id = order_product['product']['id']
        try:
            category_id = df_products[df_products['product_id'] == product_id]['category_id'].values[0]
        except:
            need_unavailable_category = True
            category_id = '00000000-0000-0000-0000-000000000000'
        
        quantity = order_product['quantity']
        price = order_product['product']['price']

        list_order_details.append({'order_id':order_id,
                                   'branch_id':branch_id,
                                   'order_date':order_date,
                                   'order_total_price':order_total_price,
                                   'order_source':order_source,
                                   'order_type':order_type,
                                   'order_status':order_status,
                                   'product_id':product_id,
                                   'category_id':category_id,
                                   'quantity':quantity,
                                   'price':price})


In [ ]:
if need_unavailable_category:
    print("Yes, we need to add the unavailable category, make sure to add it in the .env")
    new_row = pd.DataFrame({'category_id': '00000000-0000-0000-0000-000000000000', 'category_name': 'Not Available'}, index=[df_categories.index.max()+1])

    df_categories = pd.concat([df_categories, new_row], ignore_index=True)

df_categories

In [ ]:
orders_details = pd.DataFrame(list_order_details)

In [ ]:
orders_details.head()

In [ ]:
orders_details[orders_details.category_id.isna()].sort_values('order_date')

In [ ]:
cats_to_be_deleted.id.values.tolist()

In [ ]:
order_header_id_to_be_deleted = orders_details[orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())].order_id.unique()

In [ ]:
orders_details = orders_details[~orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())]

In [ ]:
orders_header = orders_header[~orders_header.order_id.isin(order_header_id_to_be_deleted)]

In [ ]:
df_products = df_products[~df_products.category_id.isin(cats_to_be_deleted.id.values.tolist())]

# UTC TO AMMAN

In [ ]:
orders_header.head()

In [ ]:
# import pytz
from datetime import timedelta

orders_details['order_date'] = pd.to_datetime(orders_details['order_date'])
# orders_details['order_date'] = orders_details['order_date'].dt.tz_localize('UTC')
# orders_details['order_date'] = orders_details['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

orders_header['order_date'] = pd.to_datetime(orders_header['order_date'])
# orders_header['order_date'] = orders_header['order_date'].dt.tz_localize('UTC')
# orders_header['order_date'] = orders_header['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

business_timezone_offset = int(os.getenv('business_timezone_offset'))
time_difference = timedelta(hours=business_timezone_offset)

orders_header['order_date'] += time_difference
orders_details['order_date'] += time_difference

In [ ]:
orders_header.head()

# Write to CSV

In [ ]:
df_products.to_csv('../../data/products.csv', index=False)

In [ ]:
orders_details.to_csv('../../data/order_details.csv', index=False)

In [ ]:
orders_header.to_csv('../../data/order_header.csv', index=False)

In [ ]:
df_categories.to_csv('../../data/categories.csv', index=False)

In [ ]:
df_branches.to_csv('../../data/branches.csv', index=False)

# Display all

In [ ]:
df_products.head()

In [ ]:
df_products.product_name.value_counts()

In [ ]:
orders_details.head()

In [ ]:
orders_header.head()